## The Contextual Bandits 

This tutorial contains a simple example of how to build a policy-gradient based agent that can solve the contextual bandit problem.

In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np

### The Contextual Bandits

Here we define our contextual bandits. In this example, we are using three four-armed bandit. What this means is that each bandit has four arms that can be pulled. Each bandit has different success probabilities for each arm, and as such requires different actions to obtain the best result. The pullBandit function generates a random number from a normal distribution with a mean of 0. The lower the bandit number, the more likely a positive reward will be returned. We want our agent to learn to always choose the bandit-arm that will most often give a positive reward, depending on the Bandit presented.

In [2]:
class contextual_bandit():
    def __init__(self):
        self.state = 0
        #list out bandits. Currently arms 4, 2, and 1 (respectively) are the most optimal
        self.bandits = np.array([[0.2,0,-0.0,-5], [0.1,-5,1,0.25],[-5,5,5,5]])
        self.num_bandits = self.bandits.shape[0] #3
        self.num_actions = self.bandits.shape[1] #4
        
    def getBandit(self):
        self.state = np.random.randint(0, len(self.bandits)) # returns a random state for each episode
        return self.state

    def pullArm(self,action):
        #get a random number
        bandit = self.bandits[self.state, action]
        result = np.random.randn(1)
        if result > bandit:
            #return a positive reward
            return 1
        else:
            #return a negative reward
            return -1

### The Policy-Based Agent

The code below established our simple neural agent. It takes as input the current state, and returns an action. This allows the agent to take actions which are conditioned on the state of the environment, a critical step toward being able to solve full RL problems. The agent uses a single set of weights, within which each value is an estimate of the value of the return from choosing a particular arm given a bandit. We use a policy gradient method to update the agent by moving the value for the selected action toward the received reward.

In [3]:
class agent():
    def __init__(self, lr, s_size, a_size):
        #these lines establish the feed-forward part of the network. The agent takes a state and produces an action
        self.state_in = tf.placeholder(shape=[1], dtype=tf.int32)
        state_in_OH = slim.one_hot_encoding(self.state_in, s_size) #shape=(1, s_size)
        output = slim.fully_connected(state_in_OH, a_size, \
                                     biases_initializer=None, activation_fn=tf.nn.sigmoid, \
                                      weights_initializer=tf.ones_initializer()) #shape=(1, a_size)
        self.output = tf.reshape(output, [-1]) #shape=(a_size,)
        self.chosen_action = tf.argmax(self.output, 0) #shape=()
        
        #the next six lines establish the training procedure. We feed the reward and chosen action into network
        #to compute the loss, and ust it to update the network
        self.reward_holder = tf.placeholder(shape=[1], dtype=tf.float32)
        self.action_holder = tf.placeholder(shape=[1], dtype=tf.int32)
        self.responsible_weight = tf.slice(self.output, self.action_holder, [1])
        self.loss = -(tf.log(self.responsible_weight)* self.reward_holder)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate = lr)
        self.update = optimizer.minimize(self.loss)

### Training the Agent

We will train our agent by getting a state from the environment, take an action, and recieve a reward. Using these three things, we can know how to properly update our network in order to more often choose actions given states that will yield the highest rewards over time.

In [4]:
tf.reset_default_graph() #clear the tensorflow graph

cBandit = contextual_bandit() #load the bandits
myAgent = agent(lr=0.001, s_size=cBandit.num_bandits, a_size=cBandit.num_actions) #load agent
weights = tf.trainable_variables()[0] #the weights to evaluate to look into the network

total_episodes = 10000 #set total number of episodes to train agent
total_reward = np.zeros([cBandit.num_bandits, cBandit.num_actions]) #set scoreboard for bandits to 0
eps = 0.1 #the chance of taking a random action

init = tf.global_variables_initializer()

In [5]:
#launch the tensorflow graph
with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        s = cBandit.getBandit() #get a state from the environment
        
        #choose either a random action or one from our network
        if np.random.rand(1) < eps:
            action = np.random.randint(cBandit.num_actions)
        else:
            action = sess.run(myAgent.chosen_action, feed_dict={myAgent.state_in: [s]})
            
        reward = cBandit.pullArm(action) #get our reward for taking an action given a bandit
        
        #update the network
        feed_dict={myAgent.reward_holder: [reward], myAgent.action_holder: [action], myAgent.state_in: [s]}
        _, ww = sess.run([myAgent.update, weights], feed_dict=feed_dict)
        
        #update the running tally of scores
        total_reward[s, action] += reward
        if i % 500 == 0:
            print("Mean reward for each of the " + str(cBandit.num_bandits) + " bandits: " + str(np.mean(total_reward,axis=1)))
        i += 1
        

Mean reward for each of the 3 bandits: [0.   0.   0.25]
Mean reward for each of the 3 bandits: [ 7.75 39.   37.  ]
Mean reward for each of the 3 bandits: [43.75 72.25 77.75]
Mean reward for each of the 3 bandits: [ 80.   117.25 108.5 ]
Mean reward for each of the 3 bandits: [115.5  152.   147.75]
Mean reward for each of the 3 bandits: [153.75 191.   183.  ]
Mean reward for each of the 3 bandits: [193.75 228.5  216.  ]
Mean reward for each of the 3 bandits: [233.   267.25 250.5 ]
Mean reward for each of the 3 bandits: [268.75 306.   287.5 ]
Mean reward for each of the 3 bandits: [307.5  339.   327.25]
Mean reward for each of the 3 bandits: [343.75 377.75 361.75]
Mean reward for each of the 3 bandits: [379.   416.25 401.5 ]
Mean reward for each of the 3 bandits: [414.5  454.   438.25]
Mean reward for each of the 3 bandits: [453.5  490.25 475.5 ]
Mean reward for each of the 3 bandits: [495.   530.   505.75]
Mean reward for each of the 3 bandits: [535.5  566.75 542.  ]
Mean reward for each

In [6]:
for a in range(cBandit.num_bandits):
    print("The agent thinks action " + str(np.argmax(ww[a])+1) + " for bandit " + str(a+1) + " is the most promising....")
    
    if np.argmax(ww[a]) == np.argmin(cBandit.bandits[a]):
        print("...and it was right!")
    else:
        print("...and it was wrong!")

The agent thinks action 4 for bandit 1 is the most promising....
...and it was right!
The agent thinks action 2 for bandit 2 is the most promising....
...and it was right!
The agent thinks action 1 for bandit 3 is the most promising....
...and it was right!
